<a href="https://colab.research.google.com/github/kaitsidis/ML102/blob/main/Pnevmonia_DensNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Εκπαίδευση Μοντέλου DenseNet με Ακτινογραφίες
μοντέλο **DenseNet** με ακτινογραφίες σε gray-scale,

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Βιβλιοθήκες που απαιτούνται**

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

Ορισμός path και φόρτωση των label
δημιουργία dictionary για την βαρύτητα των κατηγοριών

In [5]:
# Set paths for training and testing data
train_data_dir = '/content/drive/MyDrive/ML102/train_images/train_images'
test_data_dir  = '/content/drive/MyDrive/ML102/test_images_all'
labels_file    = '/content/drive/MyDrive/ML102/labels_train.csv'

# Φόρτωση του αρχείου labels
labels = pd.read_csv(labels_file)
labels['class_id'] = labels['class_id'].astype(str)  # Μετατροπή σε string για χρήση με ImageDataGenerator

# Υπολογισμός βαρύτητας ανά κατηγορία
class_weights = labels['class_id'].value_counts(normalize=True).to_dict()

### Προαιτημασία δεδομένων

In [14]:
# Δημιουργία ImageDataGenerator για εκπαίδευση και επικύρωση
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
 #   preprocessing_function=lambda x: np.pad(x, ((10, 10), (10, 10), (0, 0)), mode='constant')
    )  # Padding


train_generator = datagen.flow_from_dataframe(
    dataframe=labels,
    directory=train_data_dir,
    x_col='file_name',
    y_col='class_id',
    target_size=(224, 224),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=labels,
    directory=train_data_dir,
    x_col='file_name',
    y_col='class_id',
    target_size=(224, 224),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    subset='validation'
)

Found 3738 validated image filenames belonging to 3 classes.
Found 934 validated image filenames belonging to 3 classes.


### Δημιουργία Μοντέλου DenseNet

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, Conv2D,  Lambda
from tensorflow.keras.models import Model

input_tensor = Input(shape=(224, 224, 1))
# Wrap tf.image.grayscale_to_rgb in a Lambda layer
x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(input_tensor)

# Load pre-trained DenseNet121 with 'imagenet' weights
base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=x)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)  # 3 κατηγορίες

#Create the final model
model = Model(inputs=input_tensor, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 224, 224, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1 (Lambda)         │ (None, 224, 224, 3)    │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_4          │ (None, 230, 230, 3)    │              0 │ lambda_1[0][0]         │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,408 │ zero_padding2d_4[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_5          │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 56, 56, 64)     │              0 │ zero_padding2d_5[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 56, 56, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat  

 Total params: 7,169,091 (27.35 MB)

 Trainable params: 7,085,443 (27.03 MB)

 Non-trainable params: 83,648 (326.75 KB)

### Εκπαίδευση Μοντέλου

In [ ]:
model_file='/content/drive/MyDrive/ML102/models/densenet_model.keras' # Change the file extension to .keras

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    batch_size=32,
    class_weight=class_weights,
    verbose=1,
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(model_file, save_best_only=True)
    ]
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


### Αξιολόγηση και Οπτικοποίηση Αποτελεσμάτων

In [ ]:
# Εκτύπωση summary του μοντέλου


# Αξιολόγηση του μοντέλου
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')
# Οπτικοποίηση της ακρίβειας και της απώλειας
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Εκτύπωση τελικών αποτελεσμάτων
val_accuracy = history.history['val_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
print(f'Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}')

### Αξιολόγηση Δοκιμής

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'test_images',
    target_size=(224, 224),
    color_mode='grayscale',
    class_mode=None,
    shuffle=False
)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Αποθήκευση αποτελεσμάτων
results_df = pd.DataFrame({'file_name': test_generator.filenames, 'predicted_class': predicted_classes})
results_df.to_csv('/content/drive/MyDrive/ML102/train_csvs/test_results.csv', index=False)